# 0 开始前

本项目将用游戏 **只狼：影逝二度** 做演示。  

**注：需要具备基本的python编程能力，还要有足够的耐心看教程**  
先部分后整体，所以接下来代码会比较零散，耐心看完教程只后再看整体的代码会轻松很多。

# 1 项目基础部分

- 获取屏幕图像 grab_screen.py
- 控制键盘按键 control_keyboard_keys.py
- 检测键盘按键 detect_keyboard_keys.py

由于游戏一般不提供API，所以获取图像数据以及操作游戏就需要我们自己来解决。

## 1.1 读取游戏画面

代码：[grab_screen.py](https://github.com/ricagj/train_your_own_game_AI/blob/main/game_player/grab_screen.py)

**注：原点在左上角，所以游戏一定要对准左上角，不留间隙**

In [ ]:
# 去除边框，只留游戏画面
from game_player.grab_screen import get_full_screen
from game_player.others import get_xywh
screen = get_full_screen()
get_xywh(screen)

In [ ]:
# 测试
import cv2
from game_player.grab_screen import get_game_screen

screen = get_game_screen()
cv2.imshow('screen', screen)
cv2.waitKey(0)

In [ ]:
while True:
    screen = get_game_screen()
    cv2.imshow('screen', screen)
    cv2.waitKey(1)

## 1.2 操作游戏

代码：[control_keyboard_keys.py](https://github.com/ricagj/train_your_own_game_AI/blob/main/game_player/control_keyboard_keys.py)

control_keyboard_keys.py 内定义的函数  
~~~python
def PressKey(hexKeyCode)      # 压键
def ReleaseKey(hexKeyCode)    # 松键
~~~
完成一次按键点击，需要先执行PressKey，然后再执行ReleaseKey，相对于按下去再松开

根据游戏里设置的按键来定义执行相应动作函数  
~~~python
def W(delay=0.1):    # 移动 前
    PressKey(dk['W'])
    time.sleep(delay)

def S(delay=0.1):    # 移动 后
    PressKey(dk['S'])
    time.sleep(delay)

def A(delay=0.1):    # 移动 左
    PressKey(dk['A'])
    time.sleep(delay)

def D(delay=0.1):    # 移动 右
    PressKey(dk['D'])
    time.sleep(delay)

def ReleaseAllKey():    # 统一松开所有已定义的按键
    ReleaseKey(dk['W'])
    ReleaseKey(dk['S'])
    ReleaseKey(dk['A'])
    ReleaseKey(dk['D'])
~~~

你可以自己根据游戏设置来设置相应的按键，用记事本打开 control_keyboard_keys.py ，里面第 57 行 ~ 第 166 行提供了各种按键，然后参照上面的示例自己写出执行相应动作的函数  

In [ ]:
import time

from game_player.control_keyboard_keys import W, S, A, D

time.sleep(3)
for _ in range(5):
    print(1)
    W()
    S()
    A()
    D()

## 1.3 按键检测

代码：[detect_keyboard_keys.py](https://github.com/ricagj/train_your_own_game_AI/blob/main/game_player/detect_keyboard_keys.py)

detect_keyboard_keys.py 内定义的函数  
~~~python
def key_check()
~~~
调用它时，就会检测当前键盘上正在被按着的按键，然后把这个按键添加进列表 keys ，最后的返回值就是这个列表 keys   
不过，它只会检测我们定义好的按键，其它不会检测，比如下面定义了 **需要判断的按键** 'W', 'S', 'A', 'D', 'T', 'P' ，就只会检测这几个。  
~~~python
"""
W 移动 前
S 移动 后
A 移动 左
D 移动 右
一般我习惯用 T 控制开始，用 P 控制结束，这个不是固定的，如果和你的游戏有按键冲突，完全可以自己换一个按键
"""
def key_check():
    keys = []
    for key in ['W', 'S', 'A', 'D', 'T', 'P']:    # 需要判断的按键就是这里
        if wapi.GetAsyncKeyState(vk[key]):    # wapi.GetAsyncKeyState() ，相应的按键正在被按着，就返回 True, 否则就返回 False
            keys.append(key)    # 相应的按键正在被按着，添加进列表 keys 
    return keys
~~~
你可以自己根据游戏设置来设置相应的按键，用记事本打开 detect_keyboard_keys.py ，里面第 3 行 ~ 第 112 行提供了各种按键，然后参照上面的示例自己添加进**需要判断的按键**

In [ ]:
from game_player.detect_keyboard_keys import key_check

paused = True    # 一开始就是暂停状态，等待我们的按键信号才真正开始，
print("Ready!")

while True:

    keys = key_check()    # 死循环里会不断进行按键检测

    if paused:    # 暂停状态
        if 'T' in keys:    # 只要你按下了 T ，keys = ['T']，用 in 判断出按键信号，下一个循环就会马上进入开始状态
            paused = False
            print('\nStarting!')

    else:    # 开始状态

        print(f'\r {str(keys):<30}', end='')

        if 'P' in keys:    # 只要你按下了 P ，keys = ['P']，用 in 判断出按键信号，然后就会马上就会用 break 终止循环
            break

print('\nDone!')

## 1.4 量化状态

在本项目中，计算奖励的方法一般是根据人物状态的变化，比如目标生命值减少，自身架势增加等。由于无法直接获取这些具体数值，所以一般要通过对状态进行分析来获取。

In [ ]:
# 读取游戏画面的代码
import cv2
from game_player.grab_screen import get_game_screen

screen = get_game_screen()
cv2.imshow('screen', screen)
cv2.waitKey(0)

---

在弹出来的窗口中用鼠标左键按顺序依次点击**左下**，**左上**，**右上**，**右下**，一共 4 次，然后按键盘上的“ESC”键，就会自动返回 x, x_w, y, y_h。  
（注意：这个点击的顺序是规定好的，点击的次数也是规定好的）

In [ ]:
from game_player.others import get_xywh
get_xywh(screen)

用这个方法找到人物状态的所在图像的位置

---

把上面得到的 x, x_w, y, y_h 复制到下面

In [ ]:
from game_player.others import roi
screen_roi = roi(screen, x=402, x_w=484, y=388, y_h=390)

screen_roi 就是那部分你抠出来的图像  
对它做边缘检测，检测人物状态

In [ ]:
import numpy as np

In [ ]:
canny = cv2.Canny(cv2.GaussianBlur(screen_roi,(3,3),0), 0, 100)
value = canny.argmax(axis=-1)
print(value)
print('平均值', np.mean(value))
print('中位数', np.median(value))

观察和图像显示的会不会差别太多

**如果数值不太对，可以重新抠一次图，找到更准确的位置**

---

封装成函数

In [ ]:
def get_P(img):
    img = roi(img, x=402, x_w=484, y=388, y_h=390)
    canny = cv2.Canny(cv2.GaussianBlur(img,(3,3),0), 0, 100)
    value = canny.argmax(axis=-1)
    return np.median(value)

然后放进 others.py 指定位置
~~~python
# ---*---

# def get_P(img):
#     img = roi(img, x=402, x_w=484, y=388, y_h=390)
#     canny = cv2.Canny(cv2.GaussianBlur(img,(3,3),0), 0, 100)
#     value = canny.argmax(axis=-1)
#     return np.median(value)

def get_state_1(img):    # 自己改
    return 0

def get_state_2(img):    # 自己改
    return 0

def get_state_3(img):    # 自己改
    return 0

def get_state_4(img):    # 自己改
    return 0

# 不够就自己添加，多了就自己删除

def get_status(img):
    return get_state_1(img), get_state_2(img), get_state_3(img), get_state_4(img)    # 这里也要改成相应的函数名

# ---*---
~~~

---

In [ ]:
while True:
    screen = get_game_screen()
    print(f'\r {get_P(screen):>10}', end='')
    cv2.imshow('screen', screen)
    cv2.waitKey(1)

# 2 深度强化学习DQN基础部分之经验回放

## 参考书 **强化学习：原理与Python实现** 
[github地址](https://github.com/ZhiqingXiao/rl-book)  
![参考书](https://camo.githubusercontent.com/ab8a1a3729879574f4854e3b3f6e33ee3dc9500ce1e68e6c91e1df12b5145308/68747470733a2f2f7a686971696e677869616f2e6769746875622e696f2f696d616765732f626f6f6b2f726c2e6a7067)  

---

AI的范围有点广，在强化学习中有个更具体的名称叫Agent，也就是智能体（也有的文献叫真体）。  
- 本项目中的智能体，不是游戏里的可操作对象，而是和我们一样是个玩家。
> 智能体观测环境，获得环境的观测(observation)，记为**O**。这个过程在本项目中相当于**人类看屏幕**。  
> 智能体根据观测做出决策，决定要对环境施加的动作(action)，记为**A**。这个过程在本项目中相当于**人类做决策然后敲键盘**。  
> 环境受智能体动作的影响，改变自己的状态(state)，记为**S**，并给出奖励(reward)，记为**R**。这个过程在本项目中相当于**游戏画面发生变化然后被人类直接观测，奖励也能直接被观测**。  
>> 参考 [第一章：初识强化学习](https://anesck.github.io/M-D-R_learning_notes/RLTPI/notes_html/1.chapter_one.html)  
- 所以，智能体和我们一样，要看屏幕，要按键盘上的按键，通过屏幕上显示的状态变化例如生命值变化来判断上一瞬间自己的操作是好还是坏，并以此做相应的改进。

---

- 关于经验回放
> 经验回放（experience replay）：将经验（即历史的状态、动作、奖励等）存储起来，再按一定规则采样存储的经验。  

经验回放的作用：参考 [第六章：函数近似（function approximation）方法](https://anesck.github.io/M-D-R_learning_notes/RLTPI/notes_html/6.chapter_six.html) 四、深度 Q 学习  

## 2.1 存储经验前的数据搜集 **(S, A, R, S')**

### 2.1.1 状态 **S**(state) 与 观测 **O**(observation)

- 本项目中的 **环境、状态与观测**
    - 环境(environment)
        - 你打开游戏，操作人物在游戏里活动，游戏的地图里你所能探索、能交互的统称**环境**。注：人物也作为环境的一部分
    - 状态(state)
        - 狼、Boss、背景、UI(显示的生命值、物品栏等)等游戏显示画面里面的信息，统称**状态**。注:人物状态也作为环境状态的一部分
    - 观测(observation)
        - 你看到游戏画面并从里面获取信息的过程就叫**观测**。
        - **观测**是从**状态**里获取信息的一种手段，例如人类通过看游戏画面的方式对**状态**进行**观测**，智能体通过截屏的方式对**状态**进行**观测**。

---
为什么"经验回放"在存储的时候明明要求的是 **(S, A, R, S')** ，即 **(状态，动作，奖励，未来的状态)** ，可实际存储的却是 **(O, A, R, O')** ，即 **(观测，动作，奖励，未来的观测)** 。  
~~~python
"节选代码"
self.sekiro_agent.replayer.store(
    observation,
    action,
    reward,
    next_observation
)
~~~
因为只狼这个游戏是完全可观测的，所以观测到的结果，完全可以代表当时的状态，即 S = O, S' = O'.

- 完全可观测
    - 需要的信息，如果状态里都有，则称状态是完全可观测的，否则状态就是不可完全观测。
- 
- 例如围棋就是完全可观测的，因为双方落子位置以及整个棋盘清晰可见。
- 例如只狼这个游戏也是完全可观测的，因为对战双方的动作清晰可见，UI也显示双方的生命值和架势信息。

### 2.1.2 动作 **A**(action)

这部分代码在 **brain.py** 内

导入执行相应动作的函数
~~~python
# ---------- 以下根据 control_keyboard_keys.py 里定义的函数来导入 ----------
from game_player.control_keyboard_keys import W, S, A, D
# ---------- 以上根据 control_keyboard_keys.py 里定义的函数来导入 ----------
~~~

动作的**决策**部分和**执行**部分定义在 **智能体** 内的**choose_action**(行为选择方法)。
~~~python
# 行为选择方法
def choose_action(self, observation):

    # 先看运行的步数(self.step)有没有达到开始回放经验的要求(self.replay_start_size)，没有就随机探索
                                              # 如果已经达到了，就再看随机数在不在最终探索率范围内，在的话也是随机探索
    if self.step <= self.replay_start_size or np.random.rand() < self.min_epsilon:
        q_values = np.random.rand(self.outputs)
        self.who_play = '随机探索'
    else:
        observation = observation.reshape(-1, self.in_height, self.in_width, self.in_channels)
        q_values = self.evaluate_net.predict(observation)[0]
        self.who_play = '模型预测'

    action = np.argmax(q_values)

    # ---------- 以下根据 control_keyboard_keys.py 里定义的函数来修改 ----------

    """
    将所有的动作都用编码成数字，并且数字满足从零开始和正整数的要求。
    例如
        W 移动 前 0
        S 移动 后 1
        A 移动 左 2
        D 移动 右 3
    """

    # 执行动作
    if   action == 0:
        W()
    elif action == 1:
        S()
    elif action == 2:
        A()
    elif action == 3:
        D()
    elif action == 4:    # 等你添加，不需要可以删除
        pass
    # 不够可以添加，注意，一定要是正整数，还要和上一个相邻
    # ---------- 以上根据 control_keyboard_keys.py 里定义的函数来修改 ----------

    return action
~~~

### 2.1.3 奖励 **R**(reward)

这部分代码在 **run.py** 内

奖励部分可是强化学习的核心概念，如果你定义的奖励不能很好的赏罚分明，那智能体也就不能很好的学习。

**游戏不同，人物状态不同，请务必自己设置奖励，也只能由你自己设置奖励，没有标准答案，合理即可。**

~~~python
"节选代码"
class RewardSystem:

    # 获取奖励
    def get_reward(self, cur_status, next_status):
        """
        cur_status 和 next_status 都是存放状态信息的列表，内容：[状态1, 状态2, 状态3, 状态4]
        cur_status  表示当前的人物状态
        next_status 表示未来的人物状态
        """
        if sum(next_status) == 0:

            reward = 0
        else:
            # ---------- 以下根据 others.py 里定义的函数来修改 ----------
            # 通过列表索引的方式，取出相应的信息，用未来的状态信息减去当前的状态信息，得到状态变化值
            s1 = next_status[0] - cur_status[0]
            s2 = next_status[1] - cur_status[1]
            s3 = next_status[2] - cur_status[2]
            s4 = next_status[3] - cur_status[3]

            """
            注意，未来 - 现在
            假如你现在生命值 130（现在），过了一会生命值变成 63（未来）
            计算：s = 63 - 130, s = -67, 生命值降低了67，生命值减低应该惩罚，那么s完全可以当成得分，得到 -67 分。
            
            再假如Boss 现在生命值 112（现在）， 过了一会生命值变成 102（未来）
            计算：s = 102 - 112, s = -10, Boss生命值降低了10，应该奖励才对，但是s为负值，所以要乘上 -1 ，这样才能得到正常的分数。
            
            请根据具体的游戏来定义，不要生搬硬套，别搞得HP掉了还加分
            """
            # 示例 定义得分
            s1 *=  1    # 与 奖励 呈正相关，所以 +
            s2 *= -1    # 与 惩罚 呈正相关，所以 -
            s3 *= -1    # 与 惩罚 呈正相关，所以 -
            s4 *=  1    # 与 奖励 呈正相关，所以 +

            reward = s1 + s2 + s3 +s4
            # ---------- 以上根据 others.py 里定义的函数来修改 ----------
        return reward
~~~

# 3 模型定义

**我提供的模型是不合理的，请根据自己的项目模仿着修改**  
**我提供的模型是不合理的，请根据自己的项目模仿着修改**  
**我提供的模型是不合理的，请根据自己的项目模仿着修改**  
不懂就参考 tensorflow 官方文档  
[tf.keras.layers](https://tensorflow.google.cn/api_docs/python/tf/keras/layers)  
[tf.keras.layers.Conv2D](https://tensorflow.google.cn/api_docs/python/tf/keras/layers/Conv2D)  
[tf.keras.layers.MaxPool2D](https://tensorflow.google.cn/api_docs/python/tf/keras/layers/MaxPool2D)  
[tf.keras.layers.AveragePooling2D](https://tensorflow.google.cn/api_docs/python/tf/keras/layers/AveragePooling2D)  

这部分代码在 **brain.py** 内

~~~python
"节选代码"
# 评估网络和目标网络的构建方法
def build_network(self):
    Input = tf.keras.Input(shape=[self.in_height, self.in_width, self.in_channels])

    # -------------------- 以下务必自己修改 --------------------
    # 第 1 层 卷积层和最大池化层
    conv_1 = tf.keras.layers.Conv2D(filters=1, kernel_size=(3, 3), padding='same', activation=tf.nn.relu)(Input)
    pool_1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv_1)
    
    # 第 2 层 卷积层和最大池化层
    conv_2 = tf.keras.layers.Conv2D(filters=1, kernel_size=(3, 3), padding='same', activation=tf.nn.relu)(pool_1)
    pool_2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv_2)

    # 你要是觉得不够，可以自己增加卷积层和池化层，觉得太多了就删掉
    
    # 扁平化层
    flat = tf.keras.layers.Flatten()(pool_2)

    # 第 1 层 全连接层
    dense_1 = tf.keras.layers.Dense(1, activation=tf.nn.relu)(flat)
    dense_1 = tf.keras.layers.BatchNormalization()(dense_1)

    # 第 2 层 全连接层
    dense_2 = tf.keras.layers.Dense(1, activation=tf.nn.relu)(dense_1)
    dense_2 = tf.keras.layers.BatchNormalization()(dense_2)

    output = dense_2
    # -------------------- 以上务必自己修改 --------------------

    # 输出层
    output = tf.keras.layers.Dense(self.outputs, activation=tf.nn.softmax)(output)

    model = tf.keras.Model(inputs=Input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(self.lr),    # 你觉得有更好的可以自己改
        loss=tf.keras.losses.MeanSquaredError(),    # 你觉得有更好的可以自己改
    )

    if self.load_weights_path:
        if os.path.exists(self.load_weights_path):
            model.load_weights(self.load_weights_path)
            print('Load ' + self.load_weights_path)
        else:
            print('Nothing to load')

    return model
~~~